In [ ]:
import itertools

# Bokeh
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Legend, HoverTool
from bokeh.palettes import Set1, Category10, Category20
from bokeh.io import show, output_notebook
output_notebook()

# Line Chart

In [ ]:
import numpy as np
import pandas as pd

from bokeh.sampledata.stocks import AAPL, GOOG, MSFT, IBM

In [ ]:
df_dict = dict()
df_dict['AAPL'] = pd.DataFrame.from_dict(AAPL).astype({ 'date': np.datetime64 })
df_dict['GOOG'] = pd.DataFrame.from_dict(GOOG).astype({ 'date': np.datetime64 })
df_dict['MSFT'] = pd.DataFrame.from_dict(MSFT).astype({ 'date': np.datetime64 })
df_dict['IBM'] = pd.DataFrame.from_dict(IBM).astype({ 'date': np.datetime64 })

In [ ]:
palette = Set1
palette_colors = itertools.cycle(palette[max(palette.keys())])

fig = figure(title='Stock Closing Prices',
             x_axis_type='datetime',
             x_axis_label='Date', y_axis_label='Price',
             width=950, height=500,
             toolbar_location='above')

items = []
for key in df_dict.keys():
    df = df_dict[key]
    
    source = ColumnDataSource(data=dict(
        dates=df['date'],
        prices=df['adj_close'],
    ))
    glyph = fig.line('dates', 'prices', color=next(palette_colors), source=source)
    items.append((key, [glyph]))
    
fig.add_layout(Legend(items=items, location='bottom_left', orientation='horizontal', click_policy='hide'), 'below')
fig.add_tools(HoverTool(
    tooltips = [
        ('Date', '@dates{%F}'),
        ('Price', '@prices{0.00000}'),
    ],
    formatters={
        'dates': 'datetime',
    },
    mode='vline'
))
show(fig)

# Candlestick Chart

In [ ]:
import numpy as np
import pandas as pd

from bokeh.sampledata.stocks import AAPL

In [ ]:
df = pd.DataFrame.from_dict(AAPL).astype({ 'date': np.datetime64 })

In [ ]:
olch_indexes = df['close'] > df['open']
ohcl_indexes = df['close'] <= df['open']
candle_width = 12 * 60 * 60 * 1000 # half day in ms

fig = figure(title='Apple Stock OHLC',
             x_axis_type='datetime',
             x_axis_label='Date', y_axis_label='Price',
             width=950, height=500,
             toolbar_location='above')

items = []
labels = ['Open-Low', 'Open-High']
colors = ['#F8F8FF', '#3EF250']
dates = df['date']

glyph = fig.segment(dates, df['high'], dates, df['low'], color='black')
items.append(('High-Low', [glyph]))

for index, indexes in enumerate([olch_indexes, ohcl_indexes]):
    source = ColumnDataSource(data=dict(
        dates=[x for x in dates[indexes]],
        open=df['open'][indexes],
        close=df['close'][indexes],
        high=df['high'][indexes],
        low=df['low'][indexes],
    ))

    glyph = fig.vbar('dates', candle_width, 'open', 'close', fill_color=colors[index], line_color='black', source=source)
    items.append((labels[index], [glyph]))
    
fig.add_layout(Legend(items=items, location='bottom_left', orientation='horizontal', click_policy='hide'), 'below')
fig.add_tools(HoverTool(
    tooltips = [
        ('Date Time', '@dates{%F %H:%M:%S}'),
        ('Open', '@open{0.00000}'),
        ('High', '@high{0.00000}'),
        ('Low', '@low{0.00000}'),
        ('Close', '@close{0.00000}'),
    ],
    formatters={
        'dates': 'datetime',
    },
    mode='vline'
))
show(fig)

# Stacked Area Chart

In [ ]:
import numpy as np
import pandas as pd

from bokeh.core.properties import value

In [ ]:
N = 5
X = 10
prefix = 'Y'

data = np.random.randint(10, 100, size=(X, N))
df = pd.DataFrame(data).add_prefix(prefix)

In [ ]:
palette = Category20
palette_colors = palette[N]

fig = figure(title='Stacked Area',
             x_axis_label='X', y_axis_label='Y',
             width=950, height=500,
             toolbar_location='above')

source = ColumnDataSource(data=df)

stackers = [f'{prefix}{i}' for i in range(N)]
fig.varea_stack(stackers=stackers, x='index', color=palette_colors, legend=[value(x) for x in stackers], source=source)

fig.y_range.start = 0
fig.x_range.range_padding = 0.01

legend = fig.legend[0]
legend.visible = False
fig.add_layout(Legend(items=legend.items, location='bottom_left', orientation='horizontal', click_policy='hide'), 'below')
show(fig)

# Stacked Bar Chart

In [ ]:
from bokeh.core.properties import value

In [ ]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']
values = [
    [2, 1, 4, 3, 2, 4],
    [5, 3, 4, 2, 4, 6],
    [3, 2, 4, 4, 5, 3]
]

In [ ]:
palette = Category20
palette_colors = palette[len(years)]

fig = figure(title='Stacked Bar',
             x_axis_label='X', y_axis_label='Y',
             width=950, height=500,
             toolbar_location='above',
             tooltips=f'$name @fruits: @$name',
             x_range=fruits)

data_dict = dict()
data_dict['fruits'] = fruits
for index, year in enumerate(years):
    data_dict[year] = values[index]
    
source = ColumnDataSource(data=data_dict)
    
fig.vbar_stack(years, x='fruits', width=0.9, color=palette_colors, source=source,
               legend=[value(x) for x in years])

fig.y_range.start = 0
fig.x_range.range_padding = 0.01

legend = fig.legend[0]
legend.visible = False
fig.add_layout(Legend(items=legend.items, location='bottom_left', orientation='horizontal', click_policy='hide'), 'below')
show(fig)

# Heatmap

In [ ]:
import numpy as np
import pandas as pd

from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.sampledata.unemployment1948 import data as unemp_df
from bokeh.palettes import Reds, Greens, Blues

In [ ]:
unemp_df['Year'] = unemp_df['Year'].astype(str)
df = unemp_df.set_index('Year')
df.drop('Annual', axis=1, inplace=True)
df.columns.name = 'Month'

years = list(df.index)
months = list(df.columns)

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(df.stack(), columns=['rate']).reset_index()

In [ ]:
palette = Reds
palette_colors = np.flip(palette[max(palette.keys())])
mapper = LinearColorMapper(palette=palette_colors, low=df.rate.min(), high=df.rate.max())

fig = figure(title=f'US Unemployment ({years[0]} - {years[-1]})',
             width=950, height=500,
             toolbar_location='above',
             x_axis_location="above",
             x_range=years, y_range=np.flip(months),
             tooltips=[('date', '@Month @Year'), ('rate', '@rate%')])

fig.rect(x="Year", y="Month", width=1, height=1,
         fill_color={'field': 'rate', 'transform': mapper},
         line_color=None, source=df)

fig.axis.axis_line_color = None
fig.xaxis.major_label_orientation = np.pi / 3

color_bar = ColorBar(color_mapper=mapper,
                     major_label_text_font_size="8pt", label_standoff=8,
                     border_line_color=None, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"))
fig.add_layout(color_bar, 'right')
show(fig)

# Linked Grid Plot

In [ ]:
import numpy as np

from bokeh.layouts import gridplot

In [ ]:
N = 4
X = 10
x_range = [x for x in range(X)]
y_range = np.random.randint(10, 50, size=(1, X)).reshape(-1)

In [ ]:
palette = Category10
palette_colors = itertools.cycle(palette[max(palette.keys())])

markers = ['circle', 'circle_x', 'circle_cross', 'square', 'square_cross', 'square_x',
           'diamond', 'diamond_cross', 'triangle', 'inverted_triangle',
           'hex', 'asterisk', 'cross', 'dash', 'x']

size = 15
alpha = .5
line_color='black'

fig_dict = dict()
for index, marker in enumerate(markers):
    color = next(palette_colors)
    fig = figure(title=marker)
    
    source = ColumnDataSource(data=dict(
        x_range=x_range,
        y_range=y_range,
    ))
    
    if index == 0:
        fig.circle('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 1:
        fig.circle_x('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 2:
        fig.circle_cross('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 3:
        fig.square('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 4:
        fig.square_cross('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 5:
        fig.square_x('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 6:
        fig.diamond('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 7:
        fig.diamond_cross('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 8:
        fig.triangle('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 9:
        fig.inverted_triangle('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 10:
        fig.hex('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 11:
        fig.asterisk('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 12:
        fig.cross('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 13:
        fig.dash('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
        
    elif index == 14:
        fig.x('x_range', 'y_range', size=size, color=color, alpha=alpha, line_color=line_color, source=source)
    
    fig.add_tools(HoverTool(
        tooltips = [
            ('index', '$index'),
            ('x', '$x'),
            ('y', '$y'),
            ('x_range', '@x_range'),
            ('y_range', '@y_range'),
        ],
        mode='vline'
    ))
    fig_dict[marker] = fig


fig_list = [x for x in fig_dict.values()]
fig_list1 = fig_list[:6]
fig_list2 = fig_list[6:10]
fig_list3 = fig_list[10:]

for list_index, fig_list in enumerate([fig_list1, fig_list2, fig_list3]):
    # link plot
    for fig_index, fig in enumerate(fig_list):
        if fig_index == 0:
            continue
            
        prev_fig = fig_list[fig_index - 1]
        fig.x_range = prev_fig.x_range
        fig.y_range = prev_fig.y_range
        
    # grid plot
    ncols = 2 if list_index == 1 else 3
    grid = gridplot(fig_list, ncols=ncols, plot_width=250, plot_height=250)
    show(grid)